In [1]:
# each instance of this class stores a single fittness rule
class Fitness_Rule :  
    # takes a string which containes the rule and builds the rule
    def __init__(self, line) :
        self.option_list = []
        line_list = line.split('#')
        for element in line_list :
            if element.count(':') == 0 :
                self.option_list.append(element)
            else :
                self.option_list.append(element[:element.find(':')])
                self.fitnes_value = float(element[element.find(':')+2:])    
                
    # print overload
    def __str__(self) :
        out = ""
        for element in self.option_list :
            out += element
            out += " ** "
        out = out[:-3]
        out += " : " + str(self.fitnes_value)
        return out
        
    # takes an option dictionariy and returns the cost of this rule
    def get_partial_fitness(self, option_activation) :
        applicable = 1
        for x in self.option_list :
            applicable = applicable and option_activation[x]
        return applicable * self.fitnes_value        

    
# contains all fittness rules
class Fitness_Modell :
    # takes a string containing fittness rules and adds them to the rule list
    def add_fitness_rules(self, fittness_input) :
        for line in fittness_input :
            self.fittness_rule_list.append(Fitness_Rule(line))
            

    # takes a list containing fittness rules and builds the rule list
    def __init__(self) :
        self.fittness_rule_list = []
        
                        
    # print overload
    def __str__(self) :
        out = ""
        for element in self.fittness_rule_list :
            out += element.__str__() + "\n"
        out = out[:-2]
        return out
    
    # takes an option activation dictionary and calculates the fitness(or cost) value
    def calculate_fitness(self, option_activation) : 
        fittness = 0.0
        for element in self.fittness_rule_list :
            fittness += element.get_partial_fitness(option_activation)
        return fittness



# takes a dictionary and re
#def calculate fitness()

In [2]:
# an instance of this class stores all constrains
# TODO: optimize validity check
class Constraint_Model :
        
    def __init__(self) :
        self.constraint_list = []
        self.global_tabu_list = []
        self.mandatory_activation = []
                 
    # input line(string) list
    def build_model(self, dimacs_input) :
        self.constraint_list = []
        self.global_tabu_list = []
        self.mandatory_activation = []
        tmp_constraint = []
        
        dimacs_input = [x.strip() for x in dimacs_input]
        for line in dimacs_input :
            # fill constraint list 
            if len(line) > 0 and line[0] != 'c' and line[0] != 'p' :
                line_list = line.split(' ')
                for element in line_list :
                    if element == '0' and len(element) > 0 :
                        self.constraint_list.append(tmp_constraint)
                        tmp_constraint = []
                    elif len(element) > 0 :
                        tmp_constraint.append(int(element))      
            # initialize tabu list and mandatory activation
            elif len(line) > 0 and line[0] == 'p' :
                line_list = line.split(' ')
                self.global_tabu_list = [False] * (int(line_list[2]) + 1)
                self.mandatory_activation = [None] * (int(line_list[2]) + 1)
                
                
    # print function            
    def __str__ (self) :
        out = "Constraints:\n"
        for a in self.constraint_list :
            out += str(a) + "-"+ str(len(a)) + ", "
        
        out += "\n\nGLobal Tabu List:\n"
        out += str(self.global_tabu_list)
        
        out += "\n\nMandatory Activation List:\n"
        out += str(self.mandatory_activation)
           
        return out
        
    
    # builds the global tabu list and sets the activation values 
    def build_global_tabu_list(self) :
        new_constraint_list = []
        for constraint in self.constraint_list :  
            if len(constraint) == 1 :       
                self.global_tabu_list[abs(constraint[0])] = True      
                self.mandatory_activation[abs(constraint[0])] = constraint[0] > 0    
            else :
                new_constraint_list.append(constraint)
                
        self.constraint_list = new_constraint_list
#        for constraint in self.constraint_list :
#            if len(constraint) == 2 :
    
    
    # TODO: optimize !!!!!!
    def get_violated_variables (self, activation_arry) :
        violated_variables = []
        vio_const = []
        for constraint in self.constraint_list :
            if not self.check_partial_validity(constraint, activation_arry) :
                vio_const.append(constraint)
                for v in constraint :
                    if abs(v) not in violated_variables :
                        violated_variables.append(abs(v))
        #if len(violated_variables) < 50 :
        #    print(vio_const)
        violated_variables.sort()
        return violated_variables
    
    
    # TODO: optimize !!!!!!    
    def check_validity (self, activation_arry) :
        for constraint in self.constraint_list :
            if not self.check_partial_validity(constraint, activation_arry) :
                return False
        return True
    
    # TODO: optimize
    def check_partial_validity (self, constraint, activation_arry) :
        for x in constraint :
            if x > 0 :
                if activation_arry[x] :
                    return True
            else :
                 if not activation_arry[x * (-1)] :
                    return True   
        return False    

In [3]:
def linear_regression(n) :
    import itertools
    combinations = list(map(list, itertools.product([0, 1], repeat=n)))
    return(combinations)

def parse_xml_to_dimacs(file):
    var_map = [] #

    final_string = "" #
    variables_string = "" #
    p_line = "p cnf " #
    clauses = "" #
    current_options = [] # 
    optional_false_elements = []
    clauses_2D = []

    from lxml import etree
    xml_rules = etree.parse(open(file, 'r'))
    root = xml_rules.getroot()

    # first iteration to generate the dimacs variables
    for child in root :
        for subchild in child :
            for element in subchild : 
                if element.tag == 'name':
                    var_map.append(element.text)

    # generate variables
    for el in var_map:
        variables_string += "c " + str(1+var_map.index(el)) + " " + el + "\n"

    # generate rules
    for child in root :
        for subchild in child :
            for element in subchild :

                # temporary save current element and excluded options
                if element.tag == 'name':
                    current_options.append(element.text)
                if element.tag == 'excludedOptions' :
                    for option in element :
                        current_options.append(option.text)

                # clear temp if 'otional' tag is set to true
                if element.tag == 'optional' :
                    if element.text != 'False' :
                        current_options = []

            # if there are choices to do generate all combinations at once
            if len(current_options) > 0 :
                ###########################################################################
                # adding all combinations of rules if they are not already in 
                ###########################################################################
                # current_name, current_options
                #
                # print("Inside combinations loop", str(len(current_options)))

                if len(current_options) == 1 :
                    #print("exact one element")
                    for element in current_options:
                        #print("1")
                        tmp=[]
                        tmp.append(1+var_map.index(element))
                        clauses_2D.append(tmp)

                if len(current_options) > 1 :
                    ####################################################current_options = current_options.sort()
                    combinations = linear_regression(len(current_options))
                    for lst in combinations :
                        if not lst.count(False) == 1 :
                            tmp_list = []
                            for el, el2 in zip(lst, current_options) :
                                if el :
                                    tmp_list.append(1+var_map.index(el2))
                                else :
                                    tmp_list.append((1+var_map.index(el2))*(-1))

                            # do not put duplicate elements in list
                            tmp_list.sort()
                            is_in_list = False
                            for item in clauses_2D:
                                if item == tmp_list:
                                    is_in_list = True
                            if not is_in_list :
                                clauses_2D.append(tmp_list)

            current_options = []

    p_line += str(len(var_map))+" "+str(len(clauses_2D))+"\n"

    #print(clauses_2D)
    for line in clauses_2D :
        for item in line :
            clauses += str(item)+" "
        clauses += "0\n"

    # und zack fertig .... dimacs
    final_string = variables_string+p_line+clauses
    return final_string.split('\n')

In [15]:
class Wrapper :
    
    def __init__(self, path_to_folder, project_name) :
        from os import listdir
        from os.path import isfile, join
        
        
        self.fitness_model = Fitness_Modell()
        self.constraint_model = Constraint_Model()
        
        self.best = []
        self.popsize = 10
        self.population = []
        
       
        files = [join(path_to_folder, f) for f in listdir(path_to_folder) if isfile(join(path_to_folder, f))]
        
        for file in files:
             # -------- Fitness Model -----------
            if file.count(project_name) > 0 and (file.count('feature') > 0 or file.count('interactions') > 0) :
                with open(file, 'r') as f:
                    self.fitness_model.add_fitness_rules(f.readlines())
            
             # ------ Constraint Model ---------
            if file.count(project_name) > 0 and file.count('.xml') > 0 :
                self.constraint_model.build_model(parse_xml_to_dimacs(file))
            if file.count(project_name) > 0 and file.count('.dimacs') > 0 :
                with open(file, 'r') as f:
                    self.constraint_model.build_model(f.readlines())
                    
            # analyze constraints
            self.constraint_model.build_global_tabu_list()

            
    def init_population(self) :
        pop_tool = Population_Tool()
        
        for i in range(self.popsize) :
            individual = pop_tool.init_individual(self.constraint_model)
            print(individual)
            if individual is not None :
                self.population.append(individual)
    

In [16]:
class Population_Tool:
    
    
    def init_individual(self, constraint_model) :
        import random

        configuration = constraint_model.mandatory_activation
        tabu_config = constraint_model.global_tabu_list
        constraint_list = constraint_model.constraint_list
        
        is_valid = False
        
        counter = 0
        
        while not is_valid :
            for idx in range(len(configuration)) :
                if not tabu_config[idx]:
                    configuration[idx] = bool(random.getrandbits(1))
                    
            tmp = constraint_model.get_violated_variables(configuration)
            ptr = 0
            #print("length tmp: " + str(len(tmp)))
            for it in tmp :
                for i in range(ptr + 1, it - 1) :
                    tabu_config[i] = True
                tabu_config[it] = False
                ptr = it
            
            counter += 1
            
            if (constraint_model.check_validity(configuration) or counter > 1000) :
                is_valid = True
                return configuration
            
        return None

In [2]:
class Helper_Functions :
    # This class sontains needed functions to construct an machine learning algorithm
    
    def tweak(individual, constraint_model, fittness_model) :
        # tries to tweak an individual by randomly modify its elements 
        # (we want to stay in the valid area, so check constraints)
        
        tweaked_individual = copy.deepcopy(individual)
        
        is_valid = False
        counter = 0
        element_to_change = 0
        ind_len = len(individual)
        
        while not is_valid:
            element_to_change = random.randint(1, ind_len)
            
            if tweaked_individual[element_to_change] :
                tweaked_individual[element_to_change] = False
            else :
                tweaked_individual[element_to_change] = True
            
            if constraint_model.check_validity(tweaked_individual):
                return tweaked_individual
            else :
                # if not valid change back 
                if tweaked_individual[element_to_change] :
                    tweaked_individual[element_to_change] = False
                else :
                    tweaked_individual[element_to_change] = True
            
            counter += 1
            if counter > 1000 :
                return individual
        
        return individual
    
    
    def quality(individual, fitness_model) :
        # calgulates quality of an individual
        
        return fittness_model.calculate_fitness(individual)
        
    
    def crossover(parent1, parent2) :
        # generates two children by crossovering two parents
        
        if len(parent1) is not len(parent2) :
            print("size of both individuals should be equal to do crossover")
            return
        
        child1 = []
        child2 = []
        
        for el1, el2 in zip(parent1, parent2) :
            if bool(random.getrandbits(1)) :
                child1.append(el2)
                child2.append(el1)
            else :
                child1.append(el1)
                child2.append(el2)
        
        return child1, child2
    
    
    def diversity(individual1, individual2) :
        # compares equality of two individuals element by element and return weighted sum
        
        if len(individual1) is not len(individual2) :
            print("size of both individuals should be equal to compare")
            return
        
        elements_equal = 0
        for el1, el2 in zip(individual1, individual2) :
            if el1 == el2 :
                elements_equal += 1

        return elements_equal/len(individual1)

In [26]:
#testinstanz = Wrapper('./project_public_2/', 'busy')
#testinstanz = Wrapper('./project_public_2/', 'toy')
#testinstanz = Wrapper('./project_public_1/', 'bdbc')
testinstanz = Wrapper('./project_public_1/', 'h264')

#test_activation = [None, True, False, False, False, False, False, False, False, True, True, False, False, True, False, True, False]
testinstanz.init_population()
print(len(testinstanz.population))
#print(testinstanz.constraint_model.check_validity(test_activation))

[False, True, False, True, True, False, False, True, True, True, True, False, False, True, False, True, False]
[False, True, False, True, True, False, False, True, True, True, True, False, False, True, False, True, False]
[True, True, False, True, True, False, False, True, True, True, True, False, False, True, False, False, True]
[False, True, False, True, True, False, False, True, True, True, True, False, False, True, False, True, False]
[False, True, False, True, True, False, False, True, True, True, True, False, False, True, False, True, False]
[False, True, False, True, True, False, False, True, True, True, True, False, False, True, True, False, False]
[False, True, False, True, True, False, False, True, True, True, True, False, False, True, False, False, True]
[True, True, False, True, True, False, False, True, True, True, True, False, False, True, False, False, True]
[False, True, False, True, True, False, False, True, True, True, True, False, False, True, True, False, False]
[Fa